### import library

In [1]:
from time import time

import torch
import sklearn.datasets
import sklearn.preprocessing
import sklearn.model_selection
import numpy as np
import math

import onlinehd

### loads simple mnist dataset

In [2]:
def load():
    # fetches data
    x, y = sklearn.datasets.fetch_openml('mnist_784', return_X_y=True)
    x = x.astype(np.float)
    y = y.astype(np.int)
    y = np.array(y)

    # split and normalize
    x, x_test, y, y_test = sklearn.model_selection.train_test_split(x, y)
    scaler = sklearn.preprocessing.Normalizer().fit(x)
    x = scaler.transform(x)
    x_test = scaler.transform(x_test)

    # changes data to pytorch's tensors
    x = torch.from_numpy(x).float()
    y = torch.from_numpy(y).long()
    x_test = torch.from_numpy(x_test).float()
    y_test = torch.from_numpy(y_test).long()

    return x, x_test, y, y_test

In [3]:
# simple OnlineHD training

print('Loading...')
x, x_test, y, y_test = load()
classes = y.unique().size(0)

Loading...


### import model

In [4]:
features = x.size(1)
model = onlinehd.OnlineHD(classes, features, dim=10000)

### model.fit(x, y, encoded=False, 

### batch_size=1024, one_pass_fit=True, bootstrap=1.0, lr=0.035, epochs=20)

In [ ]:
def fit(self,
        x : torch.Tensor,
        y : torch.Tensor,
        encoded : bool = False,
        lr : float = 0.035,
        epochs : int = 120,
        batch_size : Union[int, None, float] = 1024,
        one_pass_fit : bool = True,
        bootstrap : Union[float, str] = 0.01):
    '''
    Starts learning process using datapoints `x` as input points and `y`
    as their labels.

    Args:
        x (:class:`torch.Tensor`): Input data points. Must
            have size `(n?, dim)` if `encoded=False`, otherwise must
            have size `(n?, features)`.

        encoded (bool): Specifies if input data is already encoded.

        lr (float, > 0): Learning rate.

        epochs (int, > 0): Max number of epochs allowed.

        batch_size (int, > 0 and <= n?, or float, > 0 and <= 1, or None):
            If int, the number of samples to use in each batch. If float,
            the fraction of the samples to use in each batch. If none the
            whole dataset will be used per epoch (same if used 1.0 or n?).

        one_pass_fit (bool): Whether to use onepass learning process or not.
            If true, iterative method will be used after one pass fit
            anyways for the number of epochs specified.

        bootstrap (float, > 0, <= 1 or 'single-per-class'): In order to
            initialize class hypervectors, OnlineHD does naive accumulation
            with a small fragment of data. This portion is determined by
            this argument. If 'single-per-class' is used, a single datapoint
            per class will be used as starting class hypervector.

    Warning:
        Using `one_pass_fit` is not advisable for very large data or
        while using GPU. It is expected to see high memory usage using
        this option and it does not benefit from paralellization.

    Returns:
        :class:`OnlineHD`: self
    '''

    h = x if encoded else self.encode(x)
    if one_pass_fit:
        self._one_pass_fit(h, y, lr, bootstrap)
    self._iterative_fit(h, y, lr, epochs, batch_size)
    return self

In [ ]:
# mapping in the hyper-dimension 
h = x if encoded else self.encode(x)


if one_pass_fit:
    self._one_pass_fit(h, y, lr, bootstrap)
    
self._iterative_fit(h, y, lr, epochs, batch_size)

### h : hyperdimension data

In [5]:
h = model.encoder(x)
h.shape

torch.Size([52500, 10000])

## model._one_pass_fit(h, y, lr=0.035, bootstrap=1.0)

In [6]:
cut = math.ceil(1*h.size(0))
cut

52500

In [8]:
h_ = h[:cut]
y_ = y[:cut]
h_, y_, h_.shape, y_.shape

(tensor([[-6.4814e-01, -4.8698e-01,  5.3794e-02,  ..., -4.0393e-01,
           1.6991e-02, -3.8696e-01],
         [ 1.0055e-01, -1.5533e-01,  3.6316e-01,  ..., -6.9401e-01,
           7.5901e-01,  1.2988e-01],
         [ 7.3332e-02, -3.4582e-01, -5.6562e-04,  ..., -8.6038e-01,
           4.8961e-01, -7.9107e-01],
         ...,
         [-6.8505e-01,  6.0905e-04, -2.2925e-02,  ..., -3.6865e-02,
           9.9722e-01,  1.6132e-01],
         [-1.9446e-01, -2.7448e-01,  7.4617e-01,  ...,  7.7925e-02,
           7.3011e-01, -2.0006e-01],
         [-1.8238e-01, -2.3939e-01,  4.7600e-01,  ..., -1.2340e-01,
           2.2728e-01, -7.3644e-03]]),
 tensor([5, 5, 2,  ..., 9, 6, 7]),
 torch.Size([52500, 10000]),
 torch.Size([52500]))

In [9]:
model2 = torch.zeros(10, 10000) #(classes, dim)

for lbl in range(classes):
    model2[lbl].add_(h_[y_ == lbl].sum(0), alpha=0.035)

In [10]:
banned = torch.arange(cut, device = h.device)

In [11]:
n = h.size(0)

In [12]:
todo = torch.ones(n, dtype=torch.bool, device=h.device)
todo

tensor([True, True, True,  ..., True, True, True])

In [13]:
todo[banned] = False
todo

tensor([False, False, False,  ..., False, False, False])

In [14]:
h_ = h[todo]
y_ = y[todo]
h_, y_, h_.shape, y_.shape

(tensor([], size=(0, 10000)),
 tensor([], dtype=torch.int64),
 torch.Size([0, 10000]),
 torch.Size([0]))

In [15]:
from onlinehd.onlinehd import _fasthd

_fasthd.onepass(h_, y_, model2, 0.035).shape

torch.Size([10, 10000])

In [ ]:
# will accumulate data from 0 to cut
cut = math.ceil(bootstrap * h.size(0)) # 52500 data_count
h_ = h[:cut]
y_ = y[:cut]
# updates each class hypervector (accumulating h_)

for lbl in range(self.classes):
    self.model[lbl].add_(h_[y_ == lbl].sum(0), alpha=lr)
# banned will store already seen data to avoid using it later
banned = torch.arange(cut, device=h.device)

# todo will store not used before data
n = h.size(0)
todo = torch.ones(n, dtype=torch.bool, device=h.device)
todo[banned] = False

# will execute one pass learning with data not used during model
# bootstrap
h_ = h[todo]
y_ = y[todo]
_fasthd.onepass(h_, y_, self.model, lr)

In [16]:
h_, y_, model2

(tensor([], size=(0, 10000)),
 tensor([], dtype=torch.int64),
 tensor([[ -28.3756,  -66.4885,   71.2560,  ...,  -49.9151,  114.5221,
            -3.3383],
         [ -29.8483, -134.8740,   58.2746,  ...,  -72.6994,   70.4318,
           -29.3760],
         [ -32.5078,  -91.7680,   34.5390,  ...,  -53.5381,  102.0280,
           -47.4764],
         ...,
         [ -37.9302,  -80.4323,   62.0434,  ...,  -88.7540,   79.9782,
           -74.9506],
         [ -23.0486, -106.0506,   78.0832,  ...,  -87.1586,  107.0812,
           -66.6282],
         [ -38.9152,  -86.9536,   73.6600,  ..., -105.1869,   87.9439,
           -72.4514]]))

In [ ]:
def _one_pass_fit(self, h, y, lr, bootstrap):
    # initialize class hypervectors from a single datapoint
    if bootstrap == 'single-per-class':
        # get binary mask containing whether one datapoint belongs to each
        # class
        idxs = y == torch.arange(self.classes, device=h.device).unsqueeze_(1)
        # choses first datapoint for every class
        # banned will store already seen data to avoid using it later
        banned = idxs.byte().argmax(1)
        self.model.add_(h[banned].sum(0), alpha=lr)
    else:
        # will accumulate data from 0 to cut
        cut = math.ceil(bootstrap*h.size(0))
        h_ = h[:cut]
        y_ = y[:cut]
        # updates each class hypervector (accumulating h_)
        for lbl in range(self.classes):
            self.model[lbl].add_(h_[y_ == lbl].sum(0), alpha=lr)
        # banned will store already seen data to avoid using it later
        banned = torch.arange(cut, device=h.device)

    # todo will store not used before data
    n = h.size(0)
    todo = torch.ones(n, dtype=torch.bool, device=h.device)
    todo[banned] = False

    # will execute one pass learning with data not used during model
    # bootstrap
    h_ = h[todo]
    y_ = y[todo]
    _fasthd.onepass(h_, y_, self.model, lr)

## Encoding

In [17]:
def __init__(self, features : int, dim : int = 4000):
    self.dim = dim
    self.features = features
    self.basis = torch.randn(self.dim, self.features)
    self.base = torch.empty(self.dim).uniform_(0.0, 2*math.pi)

In [28]:
basis = torch.randn(10000, 784)
basis.shape, basis

(torch.Size([10000, 784]),
 tensor([[ 0.5978,  0.8001,  0.5106,  ..., -0.3147,  0.0206,  0.8152],
         [-0.9594,  0.0663,  0.2283,  ..., -0.9433, -0.1438, -0.0430],
         [ 1.9078,  0.4669,  2.0367,  ...,  0.1945, -0.9101,  1.5179],
         ...,
         [-0.1508, -1.3020,  0.0563,  ..., -0.4242,  0.1127, -0.2668],
         [-0.0116,  0.2193,  0.1171,  ..., -0.3250,  2.3068,  0.5962],
         [-0.0846,  0.2933, -0.1665,  ..., -0.9736,  0.2808, -0.4440]]))

In [29]:
base = torch.empty(10000).uniform_(0.0, 2 * math.pi)
base.shape, base

# base`s range is 0~2*pi

(torch.Size([10000]),
 tensor([1.3664, 5.7113, 0.8909,  ..., 0.5584, 4.6792, 4.7420]))

In [20]:
def __call__(self, x:torch.Tensor):
    
    n = x.size(0)
    bsize = math.ceil(0.01*n)
    h = torch.empty(n, self.dim, device=x.device, dtype=x.dtype)
    temp = torch.empty(bsize, self.dim, device=x.device, dtype=x.dtype)

    # we need batches to remove memory usage
    for i in range(0, n, bsize):
        torch.matmul(x[i:i+bsize], self.basis.T, out=temp)
        torch.add(temp, self.base, out=h[i:i+bsize])
        h[i:i+bsize].cos_().mul_(temp.sin_())
    return h

In [30]:
# n = data size
n = x.size(0)
bsize = math.ceil(0.01*n)
n, bsize # batch_size

(52500, 525)

In [31]:
h = torch.empty(n, 10000, device=x.device, dtype=x.dtype)
h, h.shape

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 torch.Size([52500, 10000]))

In [32]:
temp = torch.empty(bsize, 10000, device=x.device, dtype=x.dtype)
temp, temp.shape

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 torch.Size([525, 10000]))

In [ ]:
# we need batches to remove memory usage
for i in range(0, n, bsize):
    torch.matmul(x[i:i+bsize], self.basis.T, out=temp)
    torch.add(temp, self.base, out=h[i:i+bsize])
#    h[i:i+bsize].cos_().mul_(temp.sin_())

In [33]:
x[0:525].shape, basis.T.shape, temp.shape

(torch.Size([525, 784]), torch.Size([784, 10000]), torch.Size([525, 10000]))

In [34]:
torch.matmul(x[0:525], basis.T, out=temp)
temp.shape, temp

(torch.Size([525, 10000]),
 tensor([[-1.9109,  0.1523,  1.0126,  ...,  0.5885, -1.7621, -0.6240],
         [-2.7255, -0.2020,  0.3480,  ...,  0.1416, -0.2177, -0.5781],
         [-0.5035, -0.1305,  0.3695,  ...,  0.1198, -0.4447,  0.1681],
         ...,
         [-1.3973,  1.2775,  0.8775,  ...,  0.0473,  0.0674, -0.8231],
         [-1.2422,  1.1375, -0.0432,  ..., -0.3398,  0.5718, -0.4027],
         [-3.6735, -0.9208, -0.9681,  ..., -1.4667, -1.2538, -0.2241]]))

In [35]:
torch.add(temp, base, out=h[0:525])
base.shape, base, h.shape, h

(torch.Size([10000]),
 tensor([1.3664, 5.7113, 0.8909,  ..., 0.5584, 4.6792, 4.7420]),
 torch.Size([52500, 10000]),
 tensor([[-0.5446,  5.8636,  1.9035,  ...,  1.1468,  2.9171,  4.1180],
         [-1.3592,  5.5093,  1.2390,  ...,  0.7000,  4.4616,  4.1640],
         [ 0.8628,  5.5807,  1.2604,  ...,  0.6782,  4.2345,  4.9102],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]))

In [36]:
h[0:525]

tensor([[-0.5446,  5.8636,  1.9035,  ...,  1.1468,  2.9171,  4.1180],
        [-1.3592,  5.5093,  1.2390,  ...,  0.7000,  4.4616,  4.1640],
        [ 0.8628,  5.5807,  1.2604,  ...,  0.6782,  4.2345,  4.9102],
        ...,
        [-0.0309,  6.9888,  1.7685,  ...,  0.6057,  4.7466,  3.9190],
        [ 0.1242,  6.8488,  0.8478,  ...,  0.2186,  5.2510,  4.3393],
        [-2.3071,  4.7905, -0.0772,  ..., -0.9083,  3.4255,  4.5180]])

In [51]:
h[0:525].cos_()

tensor([[ 0.8554,  0.9132, -0.3266,  ...,  0.4114, -0.9749, -0.5600],
        [ 0.2101,  0.7152,  0.3258,  ...,  0.7649, -0.2482, -0.5214],
        [ 0.6503,  0.7633,  0.3054,  ...,  0.7787, -0.4599,  0.1965],
        ...,
        [ 0.9995,  0.7612, -0.1964,  ...,  0.8221,  0.0342, -0.7128],
        [ 0.9923,  0.8443,  0.6616,  ...,  0.9762,  0.5130, -0.3645],
        [-0.6716,  0.0780,  0.9970,  ...,  0.6151, -0.9600, -0.1932]])

In [56]:
math.acos(0.8554)

0.5444737304601713

## Decoder

h[i:i+bsize].cos_().mul_(temp.sin_())

torch.add(temp, self.base, out=h[i:i+bsize])

torch.matmul(x[i:i+bsize], self.basis.T, out=temp)

In [53]:
h[0:525].acos()

tensor([[0.5446, 0.4196, 1.9035,  ..., 1.1468, 2.9171, 2.1651],
        [1.3592, 0.7739, 1.2390,  ..., 0.7000, 1.8216, 2.1192],
        [0.8628, 0.7024, 1.2604,  ..., 0.6782, 2.0487, 1.3730],
        ...,
        [0.0309, 0.7056, 1.7685,  ..., 0.6057, 1.5366, 2.3642],
        [0.1242, 0.5656, 0.8478,  ..., 0.2186, 1.0322, 1.9439],
        [2.3071, 1.4927, 0.0772,  ..., 0.9083, 2.8577, 1.7652]])